# Library

In [1]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import re
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [5]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

# 페이지 접속

In [6]:
# 페이지 주소
url = 'https://news.naver.com/section/101' # 네이버 뉴스 > 경제

### requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [7]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

<Response [200]>


'경제 : 네이버 뉴스'

### selenium 작동

In [8]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

### 검색 창에 검색어 입력

In [9]:
# 헤드라인 뉴스
headline_news_more_view_button_xpath = '//*[@id="newsct"]/div[1]/div[2]/a' # 헤드라인 더 보기 버튼

# 헤드라인 더 보기 버튼 로딩 대기
headline_news_more_view_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, headline_news_more_view_button_xpath))
)

# 헤드라인 더보기 버튼 클릭
headline_news_more_view_button.click()
driver.implicitly_wait(10)

## 헤드라인 뉴스 수집

In [19]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'경제 : 네이버 뉴스'

In [22]:
# 헤드라인 뉴스 섹션
headline_news_section = soup.select_one('ul.sa_list')
len(headline_news_section)

21

In [23]:
# 뉴스 타이들 리스트
headline_news_list = headline_news_section.select('li.sa_item._SECTION_HEADLINE')
len(headline_news_list)

10

### 기사 내용 섹션

In [27]:
# 기사 내용 섹션
order_number = 0
news_content_section = headline_news_list[order_number].select_one('div.sa_text')
news_content_section

<div class="sa_text">
<a class="sa_text_title _NLOG_IMPRESSION" data-clk="eco.clart" data-imp-gdid="880000D8_000000000000000015119487" data-imp-index="1" data-imp-url="https://n.news.naver.com/mnews/article/001/0015119487" href="https://n.news.naver.com/mnews/article/001/0015119487">
<strong class="sa_text_strong">전세사기·깡통전세에…세입자 '경매 셀프낙찰' 10년 만에 최대</strong>
</a>
<div class="sa_text_lede">1~12월 수도권 주택 셀프낙찰 878건…전세사기 사태 이후 급증 올해 전세 사기와 '깡통전세' 등으로 경매에 넘어간 집을 세입자가 직접 낙찰받은 사례가 10년 만에 최다였던 것으로 나타났다. 세입자 입장에서는 이미</div>
<div class="sa_text_info">
<div class="sa_text_info_left">
<div class="sa_text_press">연합뉴스</div>
<a class="sa_text_cmt _COMMENT_COUNT_LIST" data-object-id="news001,0015119487" data-processed="true" data-ticket="news" data-zero-allow="false" href="https://n.news.naver.com/mnews/article/comment/001/0015119487" style="display: none;"></a>
</div>
<div class="sa_text_info_right">
<a class="sa_text_cluster" data-clk="clcou" href="/cluster/c_202412220730_00000672/section/101?oid=001&a

In [32]:
# 기사 링크
link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
link

'https://n.news.naver.com/mnews/article/001/0015119487'

In [28]:
# 타이틀
title = news_content_section.select_one('strong.sa_text_strong').text
title

"전세사기·깡통전세에…세입자 '경매 셀프낙찰' 10년 만에 최대"

In [33]:
# 기사 내용
content = news_content_section.select_one('div.sa_text_lede').text
content

"1~12월 수도권 주택 셀프낙찰 878건…전세사기 사태 이후 급증 올해 전세 사기와 '깡통전세' 등으로 경매에 넘어간 집을 세입자가 직접 낙찰받은 사례가 10년 만에 최다였던 것으로 나타났다. 세입자 입장에서는 이미"

In [34]:
# 언론사
press = news_content_section.select_one('div.sa_text_press').text
press

'연합뉴스'

In [36]:
# 관련 뉴스 개수
related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
related_news_count

13

# 수집한 데이터 전처리

In [37]:
link_list = []
title_list = []
content_list = []
press_list = []
related_news_count_list = []

for i in range(len(headline_news_list)):
    # 기사 내용 섹션
    news_content_section = headline_news_list[i].select_one('div.sa_text')

    # 기사 링크
    try:
        link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
    except:
        link = np.nan
        print(i)
        # print('기사 링크 없음')
    
    # 타이틀
    try:
        title = news_content_section.select_one('strong.sa_text_strong').text
    except:
        title = np.nan
        print(i)
        # print('타이틀 없음')
    
    # 기사 내용
    try:
        content = news_content_section.select_one('div.sa_text_lede').text
    except:
        content = np.nan
        print(i)
        # print('기사 내용 없음')
    
    ## 언론사
    try:
        press = news_content_section.select_one('div.sa_text_press').text
    except:
        press = np.nan
        print(i)
        # print('언론사 없음')
    
    ## 관련 뉴스 개수
    try:
        related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
    except:
        related_news_count = np.nan
        print(i)
        # print('관련 뉴스 개수 없음')
    
    # 정보 리스트에 담기
    link_list.append(link)
    title_list.append(title)
    content_list.append(content)
    press_list.append(press)
    related_news_count_list.append(related_news_count)

In [38]:
len(link_list), len(title_list), len(content_list), len(press_list), len(related_news_count_list)

(10, 10, 10, 10, 10)

In [40]:
data_dict = {
    '번호': range(1, len(link_list)+1),
    '기사 링크': link_list,
    '기사 제목': title_list,
    '기사 내용': content_list,
    '언론사': press_list,
    '관련 뉴스 개수': related_news_count_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,기사 링크,기사 제목,기사 내용,언론사,관련 뉴스 개수
0,1,https://n.news.naver.com/mnews/article/001/001...,전세사기·깡통전세에…세입자 '경매 셀프낙찰' 10년 만에 최대,1~12월 수도권 주택 셀프낙찰 878건…전세사기 사태 이후 급증 올해 전세 사기와...,연합뉴스,13
1,2,https://n.news.naver.com/mnews/article/001/001...,내년 韓 제조업·수출 '경고등'…'수출주력' 반도체도 고전 예상,"산업硏·한경협·무협, 내년 전망 보고서…한목소리로 ""우려"" 반도체 등 주요 산업, ...",연합뉴스,48
2,3,https://n.news.naver.com/mnews/article/032/000...,금융자산 10억 이상 ‘부자’ 46만명···300억 이상 초부자도 첫 1만명대,지난해 국내에서 10억원 이상의 금융자산을 보유한 부자가 1% 정도 늘어 46만명을...,경향신문,47
3,4,https://n.news.naver.com/mnews/article/011/000...,"""53억 아파트 누가 샀대?""…회삿돈 끌어온 부부 정체 알고보니",#외국인 국적의 한 부부가 서울 투기과열지구 내 아파트를 53억 원에 사들였다. 남...,서울경제,41
4,5,https://n.news.naver.com/mnews/article/011/000...,개인사업자 대출 한눈에 비교한다…비교공시 서비스 개시,금융위원회는 23일부터 금융회사를 직접 방문하지 않고도 대출금리 등을 한 눈에 비교...,서울경제,17


# 데이터 출력

In [41]:
current_date = time.strftime('%Y%m%d')
current_date

'20241222'

In [42]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241222_PM_083518'

In [43]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [44]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_news', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\naver_news\\20241222'

In [45]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [46]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_news_economy_{current_datetime}.xlsx'), index=False)

In [47]:
driver.quit()

# END